<a href="https://colab.research.google.com/github/MikhailTroshin/duomo_testcase/blob/main/Scopic_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft accelerate torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# GROK

In [3]:
import json
from sklearn.model_selection import train_test_split

# Load the dataset from JSON file
with open('dataset.json', 'r') as f:
    data = json.load(f)

# Prepare data: extract code and scores
examples = []
for example_key, example in data.items():
    # Human-created code (label as NaN, we'll skip it for training)
    human_code = example['human_created']['source_code']
    examples.append({'code': human_code, 'score': None})

    # AI-generated codes with scores
    for model in ['gpt-3.5-turbo_00', 'gpt-3.5-turbo_01', 'gpt-4-turbo_00', 'gpt-4-turbo_01', 'gpt-4_00', 'gpt-4_01']:
        if model in example:
            ai_code = example[model]['source_code']
            score = float(example[model]['score']) if example[model]['score'] != 'NaN' else None
            if score is not None:
                examples.append({'code': ai_code, 'score': score})

# Filter out examples without scores (human-created)
train_data = [ex for ex in examples if ex['score'] is not None]

# Split into train (80%), validation (10%), and test (10%)
train, temp = train_test_split(train_data, train_size=0.8, random_state=42)
val, test = train_test_split(temp, train_size=0.5, random_state=42)

# Save splits to files for later use
with open('train.json', 'w') as f:
    json.dump(train, f)
with open('val.json', 'w') as f:
    json.dump(val, f)
with open('test.json', 'w') as f:
    json.dump(test, f)

print(f"Train size: {len(train)}, Val size: {len(val)}, Test size: {len(test)}")

Train size: 302, Val size: 38, Test size: 38


In [9]:
test[0]

{'code': 'def calculate_average_rating(books):\n    total_rating = 0\n    num_books = 0\n\n    for book in books:\n        total_rating += book.rating\n        num_books += 1\n\n    if num_books == 0:\n        return 0\n\n    average_rating = total_rating / num_books\n    return average_rating\n\n# Example usage\nbooks = Book.objects.all()\naverage_rating = calculate_average_rating(books)\naverage_rating\n',
 'score': 0.3}

In [4]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Add a padding token to the tokenizer
# Instead of directly assigning, use add_special_tokens
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Define tokenization function
def tokenize_function(examples):
    # Tokenize the code and add labels (scores)
    tokenized = tokenizer(examples['code'], padding="max_length", truncation=True, max_length=512)
    tokenized['labels'] = examples['score']  # Add scores as labels
    return tokenized

# Load datasets from JSON files
dataset = load_dataset('json', data_files={'train': 'train.json', 'val': 'val.json', 'test': 'test.json'})

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(['code', 'score'])
tokenized_datasets.set_format('torch')

print("Tokenization completed!")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Tokenization completed!


In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Load DistilBERT model (configured for regression with 1 output)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=1,  # Regression task
    problem_type="regression"
)

# Move model to GPU
model.to(device)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Custom Trainer to ensure eval_loss is computed and logged
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Remove labels from inputs and compute loss
        labels = inputs.pop("labels")  # Extract labels
        outputs = model(**inputs)  # Forward pass
        loss = torch.nn.functional.mse_loss(outputs.logits.squeeze(), labels)  # Compute MSE loss
        return (loss, outputs) if return_outputs else loss

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="mse",
    greater_is_better=False,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
)

# Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mse = ((predictions - labels) ** 2).mean()
    print(f"Evaluation - Predictions shape: {predictions.shape}, Labels shape: {labels.shape}, MSE: {mse}")
    return {"mse": mse}

# Custom data collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Debug: Check validation dataset
# print(f"Validation dataset size: {len(tokenized_datasets['val'])}")
# print(f"Sample from val dataset: {tokenized_datasets['val'][0]}")

# Initialize CustomTrainer with label_names
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # label_names=["labels"]  # Specify label names here for Trainer
)

# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

print("Training completed!")

TypeError: Accelerator.__init__() got an unexpected keyword argument 'use_seedable_sampler'

# QWEN

In [2]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from accelerate import Accelerator
from sklearn.metrics import mean_squared_error

In [3]:
import json
from sklearn.model_selection import train_test_split

# Load the dataset from JSON file
with open('dataset.json', 'r') as f:
    data = json.load(f)

# Prepare data: extract code and scores
examples = []
for example_key, example in data.items():
    # Human-created code (label as NaN, we'll skip it for training)
    human_code = example['human_created']['source_code']
    examples.append({'code': human_code, 'score': None})

    # AI-generated codes with scores
    for model in ['gpt-3.5-turbo_00', 'gpt-3.5-turbo_01', 'gpt-4-turbo_00', 'gpt-4-turbo_01', 'gpt-4_00', 'gpt-4_01']:
        if model in example:
            ai_code = example[model]['source_code']
            score = float(example[model]['score']) if example[model]['score'] != 'NaN' else None
            if score is not None:
                examples.append({'code': ai_code, 'score': score})

# Filter out examples without scores (human-created)
train_data = [ex for ex in examples if ex['score'] is not None]

# Split into train (80%), validation (10%), and test (10%)
train, temp = train_test_split(train_data, train_size=0.8, random_state=42)
val, test = train_test_split(temp, train_size=0.5, random_state=42)

# Save splits to files for later use
with open('train.json', 'w') as f:
    json.dump(train, f)
with open('val.json', 'w') as f:
    json.dump(val, f)
with open('test.json', 'w') as f:
    json.dump(test, f)

print(f"Train size: {len(train)}, Val size: {len(val)}, Test size: {len(test)}")

Train size: 302, Val size: 38, Test size: 38


In [5]:
# Пути к JSON файлам
train_path = "train.json"
val_path = "val.json"
test_path = "test.json"

# Создание пользовательского Dataset
class CodeDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        self.data = []
        with open(file_path, "r") as f:
            # Load the entire JSON file as a list of dictionaries
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Access the dictionary elements by key
        code = self.data[idx]["code"]
        score = self.data[idx]["score"]
        inputs = self.tokenizer(
            code,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": torch.tensor(score, dtype=torch.float32)
        }

# Инициализация токенизатора
model_name = "microsoft/codebert-base"  # Пример модели для кода
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Создание датасетов
train_dataset = CodeDataset(train_path, tokenizer)
val_dataset = CodeDataset(val_path, tokenizer)
test_dataset = CodeDataset(test_path, tokenizer)

# DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Загрузка модели для задачи регрессии
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,  # Регрессия (один выходной нейрон)
    problem_type="regression"
)

# Подготовка модели для QLoRA
model = prepare_model_for_kbit_training(model)

# Настройка LoRA конфигурации
lora_config = LoraConfig(
    r=8,  # Размерность LoRA
    lora_alpha=16,
    target_modules=["query", "value"],  # Модули, к которым применяется LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"  # Тип задачи: классификация последовательности
)

# Применение LoRA
model = get_peft_model(model, lora_config)

# Перемещение модели на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [10]:
from tqdm import tqdm

# Инициализация Accelerate
accelerator = Accelerator()
model, train_loader, val_loader = accelerator.prepare(model, train_loader, val_loader)

# Оптимизатор и планировщик
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 50
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
min_mean_mse = float('inf')

# Обучение
model.train()
for epoch in tqdm(range(num_epochs)):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            labels=batch["labels"].unsqueeze(1).to(device)
        )
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        total_loss += loss.item()

    train_mse = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {train_mse}")

    # Валидация
    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            outputs = model(
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device)
            )
            preds = outputs.logits.squeeze().cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(batch["labels"].cpu().numpy())

    val_mse = mean_squared_error(val_labels, val_preds) #, squared=False)
    print(f"Validation MSE: {val_mse}")

    curr_m_mse = (train_mse + val_mse)/2
    if curr_m_mse < min_mean_mse:
        min_mean_mse = curr_m_mse
        model.save_pretrained("fine_tuned_model")
        tokenizer.save_pretrained("fine_tuned_model")

    model.train()

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 0.057917416414343996
Validation MSE: 0.05569844652144198


  2%|▏         | 1/50 [00:23<19:34, 23.96s/it]

Epoch 2, Loss: 0.05246199415016331
Validation MSE: 0.05855848212023547


  4%|▍         | 2/50 [00:47<18:59, 23.73s/it]

...

 96%|█████████▌| 48/50 [18:26<00:45, 22.95s/it]

Validation MSE: 0.030566023646829314
Epoch 49, Loss: 0.0201447347368996


 98%|█████████▊| 49/50 [18:49<00:22, 22.92s/it]

Validation MSE: 0.03024478798008162
Epoch 50, Loss: 0.017344764268368874
Validation MSE: 0.030236558569657318


100%|██████████| 50/50 [19:13<00:00, 23.06s/it]


In [ ]:
# Сохранение модели
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")

In [11]:
# Инференс на тестовых данных
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device)
        )
        preds = outputs.logits.squeeze().cpu().numpy()
        test_preds.extend(preds)
        test_labels.extend(batch["labels"].cpu().numpy())

# Вычисление MSE на тесте
mse = mean_squared_error(test_labels, test_preds) #, squared=False)
print(f"Test RMSE: {mse}")

# Пример предикта
example_code = "def add(a, b): return a + b"
inputs = tokenizer(example_code, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
predicted_score = outputs.logits.item()
print(f"Predicted score for example code: {predicted_score}")

Test RMSE: 0.016602497824091046
Predicted score for example code: 0.7042257189750671
